In [1]:
#%pip install beautifulsoup4
#%pip install pip-system-certs

In [6]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import stardog
import os
import io
from langchain_community.graphs import Neo4jGraph
from PyPDF2 import PdfReader
from langdetect import detect
import spacy
import re
from unidecode import unidecode

Conectando ao Neo4j 

In [11]:
from neo4j import GraphDatabase

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

In [15]:
query1 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:USP'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["repository"] as repository 
"""

graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
results = graph.execute_query(query1).records
graph.close()

#results = graph.query(query2)
# Convert the results to a DataFrame
thesis_repository = pd.DataFrame(results, columns=["Thesis", "repository"])
thesis_repository = thesis_repository[1100:1200].reset_index(drop=True) 
thesis_repository

,Thesis,repository
0,tag:stardog:api:contribuicoes_linguisticas_cab...,http://www.teses.usp.br/teses/disponiveis/8/81...
1,tag:stardog:api:a_tribuna_paulista_e_a_imigrac...,https://www.teses.usp.br/teses/disponiveis/8/8...
2,tag:stardog:api:desbravando_os_sertoes_da_pied...,https://www.teses.usp.br/teses/disponiveis/8/8...
3,tag:stardog:api:escravos_e_terras_entre_posses...,https://www.teses.usp.br/teses/disponiveis/2/2...
4,tag:stardog:api:cativos_litigantes_demandas_in...,https://www.teses.usp.br/teses/disponiveis/8/8...
...,...,...
95,tag:stardog:api:a_morte_da_tradicao_a_ordem_do...,http://www.teses.usp.br/teses/disponiveis/8/81...
96,tag:stardog:api:dinamica_da_mistura_etnica_em_...,http://www.teses.usp.br/teses/disponiveis/17/1...
97,tag:stardog:api:a_economia_politica_da_violenc...,https://www.teses.usp.br/teses/disponiveis/8/8...
98,tag:stardog:api:a_atividade_artesanal_com_fibr...,http://www.teses.usp.br/teses/disponiveis/91/9...


Criando as utils functions

In [16]:

# Funcao para coletar link do pdf

def get_pdf_link(url):
    
    #Fazer requisição e parsear o arquivo html
    f = requests.get(url, verify=True).text#, verify=False) 
    soup = bs(f, "html.parser")
    
    #Coletando link para as teses
    for doc in soup.find_all('a', href=True):
        if doc['href'].endswith('.pdf'):
            path = doc['href']
            prefix_uni = 'https://www.teses.usp.br'
            link = prefix_uni + path

            return link
    
    return None

#fazer download do arquivo
def download_file(pdf_link, thesis_uri):
    # NOTE the stream=True parameter below
    r = requests.get(pdf_link, verify=True, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open('tese 2.pdf', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return 

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))


# Função para extrair texto de uma tese em PDF e identificar entidades
def extratc_text(tese_path):
    
    # Processando o arquivo em PDF
    reader = PdfReader(tese_path)
    number_of_pages = len(reader.pages)
    excerpt = {}
    

    # Iterando por cada página
    for page_number in range(number_of_pages):
        
        
        # Extraindo o texto da página
        if page_number == 0:
            page_text = (reader.pages[page_number].extract_text() + '\n ' 
                        + reader.pages[page_number+1].extract_text()[:400])
        else:
            
            if page_number == number_of_pages -1:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text())
            
            else:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text() + '\n ' 
                            + reader.pages[page_number+1].extract_text()[:400])  
      
        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
            
        except:
            lang = 'Não_detectado'
        
        # Extraindo as entidades
        page_extracted = False
        if lang == 'pt':
            doc = nlp_pt(page_text)
            page_extracted = True
        if lang == 'en':
            doc = nlp_en(page_text)
            page_extracted = True
        
        if page_extracted:

            persons = []
            gpes = []
            #orgs = []
            for ent in doc.ents:
                if ent.label_ == "PER": #"PERSON":
                    persons.append(process_uri(ent.text))
                if ent.label_ == "LOC": #"GPE":
                    gpes.append(process_uri(ent.text))
                #if ent.label_ == "ORG":
                #    orgs.append(process_uri(ent.text))
                    
            excerpt[page_number] = {'text': page_text, 
                                    'lang': lang, 
                                    'persons': list(set(persons)), 
                                    'gpes': list(set(gpes)),
                                #'orgs': list(set(orgs))
                                }

    return excerpt

Iterando por todas as URI para extrair o texto e gravar no grafo NEO4j

In [17]:
# Gravar hora de início do processamento
start_time = pd.Timestamp.now()
parcial_time = pd.Timestamp.now()

for i in range(0,len(thesis_repository)):

    # Coletando o link do PDF
    url = thesis_repository['repository'][i]
    thesis_uri = thesis_repository['Thesis'][i][16:]
    pdf_link = get_pdf_link(url)
    if pdf_link is not None:
        print(f"{i} - Downloading and extracting: {thesis_uri} from {pdf_link}")
        download_file(pdf_link, thesis_uri)
        
        # Extraindo o texto e entidades
        tese_path = 'tese 2.pdf'
        excerpt = extratc_text(tese_path)

        excerpts = []
        persons = []
        gpes = []

        # Salvando as triplas em CSV
        for n in excerpt.keys():

            excerpt_uri = thesis_repository['Thesis'][i] + '_p_' + str(n)
            excerpt_text = excerpt[n]['text']
            if "'" in excerpt_text:
                excerpt_text = excerpt_text.replace("'", "")

            excerpt_page = n
            excerpt_lang = excerpt[n]['lang']
            
            excerpts.append([thesis_repository['Thesis'][i], excerpt_uri, excerpt_text, str(excerpt_page), excerpt_lang])
                
            #Criando as relações com as entidades
            for person in excerpt[n]['persons']:
                persons.append([excerpt_uri, 'tag:stardog:api:' + person])
            
            for gpe in excerpt[n]['gpes']:
                gpes.append([excerpt_uri, 'tag:stardog:api:' + gpe])
        
        # Salvando os dados em CSV no diretório de importação do Neo4j
        pd.DataFrame(excerpts).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerpts2.csv', index=False, header=False)
        pd.DataFrame(persons).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\persons2.csv', index=False, header=False)
        pd.DataFrame(gpes).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpes2.csv', index=False, header=False)
        
        #Deletando o arquivo PDF após o processamento
        os.remove('tese 2.pdf')

        print(f"Loading triplas: {thesis_uri}")

        # Importanto os dados para o Neo4j
        #Excerpts
        query = """
        LOAD CSV FROM 'file:///excerpts2.csv'
        AS row
        MATCH  (t:Thesis{uri: row[0]})
        MERGE  (e:Exerpt{
                uri: row[1],
                text: row[2],
                page: row[3],
                lang: row[4]
                })-[r:BFO_0000050]-(t);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        #Persons
        query = """
        LOAD CSV FROM 'file:///persons2.csv'
        AS row
        MATCH  (e:Exerpt{uri: row[0]})
        MERGE  (p:Person{uri: row[1]})
        MERGE  (e)-[r:mentions]-(p);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        #GPEs
        query = """
        LOAD CSV FROM 'file:///gpes2.csv'
        AS row
        MATCH  (e:Exerpt{uri: row[0]})
        MERGE  (p:Place{uri: row[1]})
        MERGE  (e)-[r:mentions]-(p);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        # Removendo os arquivos CSV após a importação
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerpts2.csv')
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\persons2.csv')
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpes2.csv')

        print(f"Total time: {pd.Timestamp.now() - start_time}")
        print(f"Partial time: {pd.Timestamp.now() - parcial_time}")
        # Atualizando a hora parcial
        parcial_time = pd.Timestamp.now()
    else:
        print(f"No PDF link found for {thesis_uri}")

    # gravando no grafo que a tese foi processada
    query = """
    MATCH  (t:Thesis{uri: \'""" + thesis_repository['Thesis'][i] +  """\'})
    SET t.processed = true
    """
    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query)
    graph.close()

0 - Downloading and extracting: contribuicoes_linguisticas_caboverdiana_e_sefardita_na_formacao_do_papiamentu from https://www.teses.usp.br/teses/disponiveis/8/8142/tde-13102016-145726/publico/2016_ShirleyFreitasSouza_VCorr.pdf
Loading triplas: contribuicoes_linguisticas_caboverdiana_e_sefardita_na_formacao_do_papiamentu
Total time: 0 days 00:15:45.126603
Partial time: 0 days 00:15:45.126603
1 - Downloading and extracting: a_tribuna_paulista_e_a_imigracao_18351888 from https://www.teses.usp.br/teses/disponiveis/8/8138/tde-06012023-190256/publico/2022_BrunoDeSouzaPereira_VCorr.pdf
Loading triplas: a_tribuna_paulista_e_a_imigracao_18351888
Total time: 0 days 00:21:39.224636
Partial time: 0 days 00:05:54.098033
2 - Downloading and extracting: desbravando_os_sertoes_da_piedade_terra_e_trabalho_no_vale_do_paraiba_cafeeiro_bananal_c18001880 from https://www.teses.usp.br/teses/disponiveis/8/8138/tde-03102022-113831/publico/2022_BrenoAparecidoServidoneMoreno_VCorrig.pdf
Loading triplas: desbra

Transaction failed and will be retried in 1.1473605432789848s (ForsetiClient[transactionId=235482, clientId=3] can't acquire ExclusiveLock{owner=ForsetiClient[transactionId=235474, clientId=1]} on NODE_RELATIONSHIP_GROUP_DELETE(136022) because holders of that lock are waiting for ForsetiClient[transactionId=235482, clientId=3].
 Wait list:ExclusiveLock[
Client[235474] waits for [ForsetiClient[transactionId=235482, clientId=3]]])


Total time: 0 days 01:28:17.123075
Partial time: 0 days 00:12:20.446451
7 - Downloading and extracting: a_busca_da_adequacao_entre_formas_literarias_e_momento_historico_um_estudo_comparativo_entre_o_guarani_de_jose_de_alencar_e_o_escravo_de_jose_evaristo_de_almeida from https://www.teses.usp.br/teses/disponiveis/8/8156/tde-08072008-144112/publico/DISSERTACAO_FABRIZIA_DE_SOUZA_CARRIJO.pdf
Loading triplas: a_busca_da_adequacao_entre_formas_literarias_e_momento_historico_um_estudo_comparativo_entre_o_guarani_de_jose_de_alencar_e_o_escravo_de_jose_evaristo_de_almeida
Total time: 0 days 01:31:57.416225
Partial time: 0 days 00:03:40.292151
8 - Downloading and extracting: imagens_de_sombras from https://www.teses.usp.br/teses/disponiveis/27/27159/tde-05072011-125442/publico/tese.pdf
Loading triplas: imagens_de_sombras
Total time: 0 days 01:33:29.074447
Partial time: 0 days 00:01:31.658222
9 - Downloading and extracting: filologia_e_direito_na_edicao_e_analise_de_um_processo_judicial_de_execuc

IndexError: sequence index out of range